# Reinforcement learning: Tic Tac Toe et Alpha Zero

## 1/ Jeu de morpion

On va dans un premier temps créer la classe du jeu du morpion

In [1]:
import numpy as np

class Morpion_board():
    def __init__(self):
        self.board = np.zeros([3, 3]).astype(str)
        self.board[self.board == "0.0"] = ' '
        self.player = 1

    def draw_sign(self, move):
        if hasattr(move, "__len__"):
            row, column = move[0], move[1]
        else:
            row = move//3
            column = move%3
        if self.board[row, column] != ' ':
            return ('Invalid position')
        else:
            if self.player == 0:
                self.board[row, column] = 'O'
                self.player = 1
            elif self.player == 1:
                self.board[row, column] = 'X'
                self.player = 0

    def check_winner(self):
        if self.player == 0:
            # on essaye les 6 points qui permettent de gagner (pour le joueur 0)
            if self.board[0, 0] == 'O':
                if (self.board[0, 0] == 'O' and self.board[0, 1] == 'O' and self.board[
                    0, 2] == 'O'):
                    return True
                elif (self.board[0, 0] == 'O' and self.board[1, 0] == 'O' and self.board[
                    2, 0] == 'O'):
                    return True
                elif (self.board[0, 0] == 'O' and self.board[1, 1] == 'O' and self.board[
                    2, 2] == 'O'):
                    return True
            if self.board[1, 1] == 'O':
                if (self.board[0, 1] == 'O' and self.board[1, 1] == 'O' and self.board[
                    2, 1] == 'O'):
                    return True
                elif (self.board[1, 0] == 'O' and self.board[1, 1] == 'O' and self.board[
                    1, 2] == 'O'):
                    return True
                elif (self.board[2, 0] == 'O' and self.board[1, 1] == 'O' and self.board[
                    0, 2] == 'O'):
                    return True
            if self.board[2, 2] == 'O':
                if (self.board[0, 2] == 'O' and self.board[1, 2] == 'O' and self.board[
                    2, 2] == 'O'):
                    return True
                elif (self.board[2, 0] == 'O' and self.board[2, 1] == 'O' and self.board[
                    2, 2] == 'O'):
                    return True

        if self.player == 1:
            # on essaye les 6 points qui permettent de gagner (pour le joueur 1)
            if self.board[0, 0] == 'X':
                if (self.board[0, 0] == 'X' and self.board[0, 1] == 'X' and self.board[
                    0, 2] == 'X'):
                    return True
                elif (self.board[0, 0] == 'X' and self.board[1, 0] == 'X' and self.board[
                    2, 0] == 'X'):
                    return True
                elif (self.board[0, 0] == 'X' and self.board[1, 1] == 'X' and self.board[
                    2, 2] == 'X'):
                    return True
            if self.board[1, 1] == 'X':
                if (self.board[0, 1] == 'X' and self.board[1, 1] == 'X' and self.board[
                    2, 1] == 'X'):
                    return True
                elif (self.board[1, 0] == 'X' and self.board[1, 1] == 'X' and self.board[
                    1, 2] == 'X'):
                    return True
                elif (self.board[2, 0] == 'X' and self.board[1, 1] == 'X' and self.board[
                    0, 2] == 'X'):
                    return True
            if self.board[2, 2] == 'X':
                if (self.board[0, 2] == 'X' and self.board[1, 2] == 'X' and self.board[
                    2, 2] == 'X'):
                    return True
                elif (self.board[2, 0] == 'X' and self.board[2, 1] == 'X' and self.board[
                    2, 2] == 'X'):
                    return True
        return False

    def actions(self, as_1d=False):# returns all possible moves
        acts = []
        for row in range(3):
            for col in range(3):
                if self.board[row, col] == ' ':
                    acts.append([row, col])
        if as_1d:
            for i in range(len(acts)):
                acts[i] = acts[i][0]*3+acts[i][1]
        return acts

    def __repr__(self):
        return(f"""|{self.board[0,0]}|{self.board[0,1]}|{self.board[0,2]}|
|{self.board[1,0]}|{self.board[1,1]}|{self.board[1,2]}|
|{self.board[2,0]}|{self.board[2,1]}|{self.board[2,2]}|""")


On va maintenant vérifier si ce dernier fonctionne.

In [2]:
M = Morpion_board()
print(M)
M.draw_sign([1,2])
print(M)
print(M.actions(True))
print(M.check_winner())

| | | |
| | | |
| | | |
| | | |
| | |X|
| | | |
[0, 1, 2, 3, 4, 6, 7, 8]
False


## Les 3 éléments d'Alpha Zéro

- Un réseau de neurone adapté au board qui renvoie la policy (distribution en proba des différents mouvements possibles) ainsi que du vainqueur 
- Monte-Carlo Tree Search (MCTS): comme Alpha Chess Zéro, nous avons implémenté la méthode MCTS plutôt que alpha beta.  On utilise la policy pour générer un ensemble de jeu.
- Evaluation du réseau de neurone, basé sur une évaluation du gagnant du match à la prochaine itération

## Le réseau de neurone 

On cherche dans un premier temps à avoir un plateau de jeu intelligible pour notre neural network. On va donc transformer notre plateau de jeu en 1 dimension en trois dimension. 
![baba](img/board_to_neural.png)

On va donc d'abord définir la fonction encode board, pour passer d'un board à un array numpy.

In [3]:
def encode_board(board):
    board_sate = board.board
    encoded = np.zeros([3,3,3]).astype(int)
    encoder_dict = {'O':1, 'X':0}
    for row in range(3):
        for col in range(3):
            if board_sate[row,col] != ' ':
                encoded[row, col, encoder_dict[board_sate[row, col]]] = 1
    if board.player == 1:
        encoded[:,:,2] = 1
    return encoded

On teste si elle fonctionne 

In [4]:
encoded_board = encode_board(M)
print(encoded_board[:,:,0])
print(encoded_board[:,:,1])
print(encoded_board[:,:,2])
print(f"The player that is about to play is {M.player}")

[[0 0 0]
 [0 0 1]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
The player that is about to play is 0


On va maintenant aussi coder une fonction decode board, pour passer d'un array numpy à un board state.

In [5]:
def decode_board(encoded):
    decoded = np.zeros([3,3]).astype(str)
    decoded[decoded == '0.0'] = ' '
    decoder_dict = {0:'O', 1:'X'}
    for row in range(3):
        for col in range(3):
            for k in range(2):
                if encoded[row,col,k] == 1:
                    decoded[row,col] = decoder_dict[k]
    mboard = Morpion_board()
    mboard.board = decoded
    mboard.player = encoded[0,0,2]
    return mboard

In [6]:
decoded_board = decode_board(encoded_board)
print(decoded_board)

| | | |
| | |O|
| | | |


On retrouve bien notre plateau initial

## Architecture de notre réseau de neurone

L'architecture d'Alpha Chess est assez simple. Il y a dans un premier temps un premier network de convolution, suivi d'un deuxième dit résiduel. Avant enfin d'obtenir le réseau d'output qui renvoie la policy, c'est à dire l'ensemble des probabilités de choix de prochain move et la value, c'est à dire le vainqueur. 

On a représenté le réseau de la façon suivante.
![babi](img/neural_net.png)

En nous inspirant de l'infrastructure réalisée pour AlphaChess Zero nous avons définit en pytorch le réseau suivant.

### Convolutionnal block

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
import matplotlib.pyplot as plt
import numpy as np

class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock,self).__init__()
        self.action_size = 3
        self.conv1 = nn.Conv2d(3,128,3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(128)

    def forward(self, s):
        s = s.view(-1,3,3,3) #reshape the tensor
        s = F.relu(self.bn1(self.conv1(s)))
        return s

### Residual block


Nous n'avons ici détaillé qu'un seul des 19 residuals blocks utilisé dans le cas d'AlphaChess. Par ailleurs nous avons diminué leur nombre.

In [8]:
class ResBlock(nn.Module):
    def __init__(self, inplanes=128, planes=128, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = F.relu(out)
        return out

### Output block

Notre block d'output nous donne notre policy $p$ et notre vainqueur $v$.

In [9]:
class OutBlock(nn.Module):
    def __init__(self):
        super(OutBlock, self).__init__()
        self.conv = nn.Conv2d(128, 3, kernel_size=1)
        self.bn = nn.BatchNorm2d(3)
        self.fc1 = nn.Linear(3*3*3, 32)
        self.fc2 = nn.Linear(32,1)

        self.conv1 = nn.Conv2d(128, 32, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        self.fc = nn.Linear(3*3*32, 9)

    def forward(self, s):
        v = self.conv(s)
        v = self.bn(v)
        v = F.relu(v)
        v = v.view(-1, 3*3*3)
        v = self.fc1(v)
        v = F.relu(v)
        v = self.fc2(v)
        v = F.tanh(v)

        p = self.conv1(s)
        p = self.bn1(p)
        p = F.relu(p)
        p = p.view(-1, 3*3*32)
        p = self.fc(p)
        p = self.logsoftmax(p).exp()
        return p, v

### Notre réseau complet

On peut donc enfin définir notre réseau complet

In [10]:
class ConnectNet(nn.Module):
    def __init__(self):
        super(ConnectNet,self).__init__()
        self.conv = ConvBlock()
        for block in range(5):
            setattr(self, 'res%i' % block, ResBlock())
        self.outblock = OutBlock()

    def forward(self, s):
        s = self.conv(s)
        for block in range(5):
            s = getattr(self, 'res%i' % block)(s)

        s = self.outblock(s)
        return s

### Test de notre modèle

In [11]:
con = ConnectNet()
m = torch.from_numpy(encoded_board).float()
con(m)

/usr/local/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


(tensor([[0.0528, 0.0789, 0.1466, 0.1009, 0.0611, 0.0642, 0.2066, 0.1446, 0.1443]],
        grad_fn=<ExpBackward>), tensor([[0.0012]], grad_fn=<TanhBackward>))

On obtient bien un array de la taille attendue. La policy est un array de taille $9$ et la valeur de taille $1$.

### Definir la loss

On doit définir la loss de notre modèle à deux output. On va définir une loss personnalisée TicLoss comme une somme d'une erreur quadratique moyenne et d'une cross entropy, permettant de rendre compte des deux composantes de notre output.

In [12]:
class TicLoss(torch.nn.Module):
    def __init__(self):
        super(TicLoss, self).__init__()

    def forward(self, y_value, value, y_policy, policy):
        value_error = (value - y_value) ** 2
        policy_error = torch.sum((-policy *
                                  (1e-8 + y_policy.float()).float().log()), 1)
        total_error = (value_error.view(-1).float() + policy_error).mean()
        return total_error

## Recherche avec Arbre Monte Carlo (Monte Carlo Tree Search)

On va maintenant implémenter la partie MCTS.

Un jeu est assimilé à un arbre dans lequel la racine est le plateau et les différentes branches tous les états possibles qui peuvent en résulter. Il est facile de calculer tous l'arbre complet du jeu dans le cas du morpion. Ce n'est pas le cas pour un jeu comme les échecs, car le nombre de possibilités augmente de façon exponentielle. 

On va donc utiliser l'algorithme Monte-Carlo Tree Search pour restreindre notre arbre à ses composantes intéressantes.

On se retrouve dans un cas classique de bandit, dans lequel on doit maximiser notre compromis exploration-exploitation. Exploration car on veut observer la plupart des branches, exploitations car on veut se concentrer sur les seules branches intéressantes.

Ceci est décrit succinctement dans une équation de l'algorithme MCTS qui définit la limite supérieure de confiance (Upper Confidence Bound) :

$$Q + cP(s,a)\times \frac{\sqrt{\sum_{b}N(s,b)}}{1+N(s,a)}$$

- $Q$ est la valeur moyenne de l'action
- $c$ est une constante déterminant le niveau d'exploration (fixée à 1)
- $P(s=état,a=action)$ est la probabilité préalable de choisir l'action a donnée par la sortie de la policy du neural network
- $N(s,a)$ est le nombre de fois que la branche correspondant à l'action $a$ a été visitée
- $\sum_{b}N(s,b)$ le nombre de fois que le parent de $(s,a)$ a été visité.

### Les différentes étapes de l'algorithme MCTS

![babi](img/mcts.png)

#### Sélection

![babi](img/selection.png)

On commence d'abord une première étape de sélection. En partant de l'état $s$, on choisit la branche suivante ayant le UCB le plus haut dans un noeud feuille (jamais exploré), ou noeud terminal (fin du jeu). Notre réseau de neurones guide la sélection en donnant la probabilité $P$, qui est pour l'instant aléatoire, notre réseau n'est pas entrainé.

#### Expansion et évaluation

![babi](img/expansion.png)


Dans cette phase, la feuille est développée en évaluant les états associés aux noeuds enfants grâce au réseau et stocker $P$. De manière classique nous avons ajouté un bruit de Dirichler sur le noeud racine pour améliorer notre exploration. 


#### Backup

![babi](img/backup.png)

Maintenant, le noeud feuille est évalué par le réseau neuronal pour déterminer sa valeur $v$. Cette valeur est ensuite utilisée pour mettre à jour la moyenne $v$ de tous les nœuds parents au-dessus de lui. Si O gagne ($v = +1$ évalué pour le noeud feuille), alors dans le noeud parent direct de ce noeud feuille, ce sera au tour de O de jouer et nous mettrons à jour $v = +1$ pour ce noeud parent, puis nous mettrons à jour $v = -1$ pour tous les autres noeuds parents où X doit jouer pour indiquer que cette action est mauvaise pour X. Enfin, nous mettrons à jour $v = 0$ en cas d'égalité.


La classe obtenue pour coder cet algorithme est très grande, mais on retrouve les différentes étapes.

In [13]:
import torch.multiprocessing as mp
import copy
import math
import datetime

class UCTNode():
    def __init__(self, game, move, parent=None):
        self.game = game  # state s
        self.move = move  # action index
        self.is_expanded = False
        self.parent = parent
        self.children = {}
        self.child_priors = np.zeros([9], dtype=np.float32)
        self.child_total_value = np.zeros([9], dtype=np.float32)
        self.child_number_visits = np.zeros([9], dtype=np.float32)
        self.action_idxes = []

    @property
    def number_visits(self):
        return self.parent.child_number_visits[self.move]

    @number_visits.setter
    def number_visits(self, value):
        self.parent.child_number_visits[self.move] = value

    @property
    def total_value(self):
        return self.parent.child_total_value[self.move]

    @total_value.setter
    def total_value(self, value):
        self.parent.child_total_value[self.move] = value

    def child_Q(self):
        return self.child_total_value / (1 + self.child_number_visits)

    def child_U(self):
        return math.sqrt(self.number_visits) * (
                abs(self.child_priors) / (1 + self.child_number_visits))

    def best_child(self):
        if self.action_idxes != []:
            bestmove = self.child_Q() + self.child_U()
            bestmove = self.action_idxes[np.argmax(bestmove[self.action_idxes])]
        else:
            bestmove = np.argmax(self.child_Q() + self.child_U())
        return bestmove

    def select_leaf(self):
        current = self
        while current.is_expanded:
            best_move = current.best_child()
            current = current.maybe_add_child(best_move)
        return current

    def add_dirichlet_noise(self, action_idxs, child_priors):
        valid_child_priors = child_priors[action_idxs]  # select only legal moves entries in child_priors array
        valid_child_priors = 0.75 * valid_child_priors + 0.25 * np.random.dirichlet(np.zeros([len(valid_child_priors)],dtype=np.float32) + 192)
        child_priors[action_idxs] = valid_child_priors
        return child_priors

    def expand(self, child_priors):
        self.is_expanded = True
        action_idxs = self.game.actions(as_1d=True);
        c_p = child_priors
        if action_idxs == []:
            self.is_expanded = False
        self.action_idxes = action_idxs
        c_p[[i for i in range(len(child_priors)) if i not in action_idxs]] = 0.000000000  # mask all illegal actions
        if self.parent.parent == None:  # add dirichlet noise to child_priors in root node
            c_p = self.add_dirichlet_noise(action_idxs, c_p)
        self.child_priors = c_p

    def decode_n_move_pieces(self, board, move):
        board.draw_sign(move)
        return board

    def maybe_add_child(self, move):
        if move not in self.children:
            copy_board = copy.deepcopy(self.game)  # make copy of board
            copy_board = self.decode_n_move_pieces(copy_board, move)
            self.children[move] = UCTNode(
                copy_board, move, parent=self)
        return self.children[move]

    def backup(self, value_estimate: float):
        current = self
        while current.parent is not None:
            current.number_visits += 1
            if current.game.player:  # same as current.parent.game.player = 0
                current.total_value += (1 * value_estimate)  # value estimate +1 = O wins
            elif not current.game.player:  # same as current.parent.game.player = 1
                current.total_value += (-1 * value_estimate)
            current = current.parent

In [14]:
class DummyNode(object):
    def __init__(self):
        self.parent = None
        self.child_total_value = collections.defaultdict(float)
        self.child_number_visits = collections.defaultdict(float)
        
def do_decode_n_move_pieces(board,move):
    if type(move)==int:
        row = move//3
        col = move%3
        board.draw_sign([row, col])
    else:
        board.draw_sign(move)
    return board

#### Run

In [15]:
def UCT_search(game_state, num_reads,net,temp):
    root = UCTNode(game_state, move=None, parent=DummyNode())
    for i in range(num_reads):
        leaf = root.select_leaf()
        encoded_s = encode_board(leaf.game)
        encoded_s = torch.from_numpy(encoded_s).float()
        child_priors, value_estimate = net(encoded_s)
        child_priors = child_priors.detach().cpu().numpy().reshape(-1)
        value_estimate = value_estimate.item()
        if leaf.game.check_winner() == True or leaf.game.actions() == []: # if somebody won or draw
            leaf.backup(value_estimate); continue
        leaf.expand(child_priors) # need to make sure valid moves
        leaf.backup(value_estimate)
    return root


Le processus de sélection, d'expansion et d'évaluation et de sauvegarde représente une simulation pour chaque nœud racine de l'algorithme des SCTM. Dans AlphaChess Zero, 800 simulations de ce type sont effectuées. 

Pour notre implémentation du morpion, nous n'en avons effectué que 50 car c'est un jeu beaucoup plus simple. Après avoir exécuté 100 simulations pour ce nœud racine, nous formulerons la politique $p$ pour le nœud racine qui est définie comme étant proportionnelle au nombre de visites de ses nœuds enfants directs. Cette politique $p$ sera ensuite utilisée pour sélectionner le prochain coup vers le prochain état du plateau, et cet état du plateau sera alors traité comme le nœud racine pour les prochaines simulations de SCTM et ainsi de suite jusqu'à ce que le jeu se termine lorsque quelqu'un gagne ou fait match nul. 

Toute la procédure dans laquelle on exécute des simulations de SCTM pour chaque nœud racine au fur et à mesure des déplacements jusqu'à la fin du jeu est appelée <b>"self-play MCTS"</b>.
    
On va dans un premier temps créer des fonctions utilitaires pour enregistrer les différentes itérations.

In [16]:
import pickle
import collections

def save_as_pickle(filename, data):
    complete_name = os.path.join("./datasets/", filename)
    with open(complete_name, 'wb') as output:
        pickle.dump(data, output)

def load_pickle(filename):
    complete_name = os.path.join("./datasets/", filename)
    with open(complete_name, 'rb') as pkl_file:
        data = pickle.load(pkl_file)
    return data

def get_policy(root, temp=1):
    policy = ((root.child_number_visits) ** (1 / temp)) / sum(root.child_number_visits ** (1 / temp))
    return policy

In [17]:
def MCTS_self_play(connectnet,num_games, start_idx, cpu, args, iteration):
    for idxx in range(0,num_games):
        current_board = Morpion_board()
        checkmate = False
        dataset = [] # to store state, policy, value for neural network training
        states = []; value = 0; move_count = 0
        # play game against self
        while checkmate == False and current_board.actions() != []:
            # set temperature parameter
            if move_count < 11:
                t = 1
            else:
                t = 0.1
            states.append(copy.deepcopy(current_board.board))
            board_state = copy.deepcopy(encode_board(current_board))
            root = UCT_search(current_board,50,connectnet,t) # run 777 MCTS simulations
            policy = get_policy(root, t) # formulate policy based on results of MCTS simulations
            current_board = do_decode_n_move_pieces(current_board,\
                                                    np.random.choice(np.array([0,1,2,3,4,5,6,7,8]), \
                                                                     p = policy)) # decode action and make a move
            dataset.append([board_state,policy]) # stores s, p
            if current_board.check_winner() == True: # if somebody won, update v
                if current_board.player == 0: # X wins
                    value = -1
                elif current_board.player == 1: # O wins
                    value = 1
                checkmate = True
            move_count += 1
        dataset_p = []
        # update v for all (s, p) except for starting board state s
        for idx,data in enumerate(dataset):
            s,p = data
            if idx == 0:
                dataset_p.append([s,p,0])
            else:
                dataset_p.append([s,p,value])
        del dataset
        # save (s,p,v) datasets for neural net training
        if not os.path.isdir("./datasets/iter_%d/" % iteration):
            os.mkdir("./datasets/iter_%d/" % iteration)
        save_as_pickle("iter_%d/" % iteration + "dataset_cpu%i_%i_%s" % (cpu,idxx, datetime.datetime.today().strftime("%Y-%m-%d")),dataset_p)

In [18]:
from logzero import logger

def run_MCTS(args, start_idx=0, iteration=0):
    net_to_play = f"{args.neural_net_name}_iter{iteration}.pth.tar"
    net = ConnectNet()
    cuda = torch.cuda.is_available()
    
    
    logger.info('Preparing model for MCTS...')
    net.eval()

    current_net_filename = os.path.join('./model_data/', net_to_play)

    if os.path.isfile(current_net_filename):
        checkpoint = torch.load(current_net_filename)
        net.load_state_dict(checkpoint['state_dict'])
        logger.info('Loaded %s model.' % current_net_filename)
    else:
        torch.save({'state_dict': net.state_dict()}, os.path.join("./model_data/",net_to_play))
        logger.info('Initialized model.')

    with torch.no_grad():
        MCTS_self_play(net, args.num_games_per_MCTS_process, start_idx, 0, args, iteration)
    logger.info('Finished MCTS!')

In [19]:
import plotly.express as px

def train(net, dataset, optimizer, scheduler, start_epoch, cpu, args, iteration):
    torch.manual_seed(cpu)
    cuda = torch.cuda.is_available()
    net.train()
    criterion = TicLoss()
    train_set = board_data(dataset)
    train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=0, pin_memory=False)
    losses_per_epoch = load_results(iteration + 1)

    logger.info("Starting training process...")
    print(f"the length of the train loader is{len(train_loader)}")
    update_size = max(len(train_loader)//10,1)
    print("Update step size: %d" % update_size)
    for epoch in range(start_epoch, args.num_epochs):
        total_loss = 0.0
        losses_per_batch = []
        for i, data in enumerate(train_loader, 0):
            state, policy, value = data
            state = state.float()
            policy = policy.float()
            value = value.float()
            if cuda:
                state = state.cuda()
                policy = policy.cuda()
                value = value.cuda()
            policy_pred, value_pred = net(state)
            loss = criterion(value_pred[:,0], value, policy_pred, policy)
            loss = loss/args.gradient_acc_steps
            loss.backward()
            clip_grad_norm_(net.parameters(), args.max_norm)
            if (epoch % args.gradient_acc_steps) == 0:
                optimizer.step()
                optimizer.zero_grad()

            total_loss += loss.item()
            print(total_loss)
            print("Update step size: %d" % update_size)
            if i % update_size == (update_size - 1):
                losses_per_batch.append(args.gradient_acc_steps*total_loss/update_size)
                print('[Iteration %d] Process ID: %d [Epoch: %d, %5d/ %d points] total loss per batch: %.3f' %
                      (iteration, os.getpid(), epoch + 1, (i + 1) * args.batch_size, len(train_set),
                       losses_per_batch[-1]))
                print("Policy (actual, predicted):", policy[0].argmax().item(), policy_pred[0].argmax().item())
                print("Policy data:", policy[0]);
                print("Policy pred:", policy_pred[0])
                print("Value (actual, predicted):", value[0].item(), value_pred[0, 0].item())
                print(" ")
                total_loss = 0.0

        scheduler.step()
        if len(losses_per_batch) >=1:
            losses_per_epoch.append(sum(losses_per_batch)/len(losses_per_batch))
        if (epoch % 2) == 0:
            save_as_pickle("losses_per_epoch_iter%d.pkl" % (iteration + 1), losses_per_epoch)
            torch.save({
                'epoch' : epoch +1,
                'state_dict': net.state_dict(),
                'optimizer' : optimizer.state_dict(),
                'scheduler' : scheduler.state_dict(),
                'loss':loss
            }, os.path.join(f'./model_data/',f"{args.neural_net_name}_iter{(iteration+1)}.pth.tar"))

    logger.info('Finished Training!')
    fig = px.scatter([e for e in range(start_epoch, (len(losses_per_epoch) + start_epoch))], loss)
    fig.update_layout(
        title="loss vs Epoch",
        xaxis_title="Epoch",
        yaxis_title="Loss per batch",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.write_image(os.path.join("./model_data/", "Loss_vs_Epoch_iter%d_%s.png" % (
    (iteration + 1), datetime.datetime.today().strftime("%Y-%m-%d"))))
    
    
    

def train_connectnet(args, iteration, new_optim_state):
    logger.info("Loading training data...")
    data_path = "./datasets/iter_%d/" % iteration
    datasets = []
    for idx, file in enumerate(os.listdir(data_path)):
        filename = os.path.join(data_path, file)
        with open(filename, 'rb') as fo:
            datasets.extend(pickle.load(fo, encoding='bytes'))
    datasets = np.array(datasets)
    logger.info("Loaded data from %s." % data_path)
    net = ConnectNet()
    cuda = torch.cuda.is_available()
    if cuda:
        net.cuda()
    optimizer = optim.Adam(net.parameters(), lr=args.lr, betas=(0.8, 0.999))
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,100,150, 200, 250, 300, 400], gamma=0.77)
    start_epoch = load_state(net, optimizer, scheduler, args, iteration, new_optim_state)

    train(net, datasets, optimizer, scheduler, start_epoch, 0, args, iteration)

### Entrainement


In [20]:
class arena():
    def __init__(self, current_cnet, best_cnet):
        self.current = current_cnet
        self.best = best_cnet


    def play_round(self):
        logger.info("starting game round...")
        if np.random.uniform(0,1) <= 0.5:
            white = self.current
            black = self.best
            w = "current"
            b = "best"
        else:
            white = self.best
            black = self.current
            w = 'best'
            b= 'current'
        current_board = Morpion_board()
        checkmate = False
        dataset = []
        value = 0
        t = 0.1
        while checkmate == False and current_board.actions() != []:
            dataset.append(copy.deepcopy(encode_board(current_board)))
            if current_board.player == 0:
                root = UCT_search(current_board, 50, white, t)
                policy = get_policy(root, t)
                print("Policy : ", policy, "white = %s" %(str(w)))
            elif current_board.player == 1:
                root = UCT_search(current_board, 50, black, t)
                policy = get_policy(root, t)
                print("Policy : ", policy, "black = %s" % (str(b)))
            current_board = do_decode_n_move_pieces(current_board, np.random.choice(np.array([0,1,2,3,4,5,6,7,8]), p = policy))
            if current_board.check_winner() == True:
                if current_board.player == 0:
                    value = -1
                elif current_board.player == 1:
                    value = 1
                checkmate = True
        dataset.append(encode_board(current_board))
        if value == -1:
            dataset.append(f"{b} as circle wins")
            return b, dataset
        elif value == 1:
            dataset.append(f"{w} as cross wins")
            return w, dataset
        else:
            dataset.append('Nobody wins')
            return None, dataset
    def evaluate(self, num_games, cpu):
        current_wins = 0
        logger.info("[CPU %d]: Starting games..." % cpu)
        for i in range(num_games):
            with torch.no_grad():
                winner, dataset = self.play_round()
                print("%s wins!" % winner)
            if winner == 'current':
                current_wins += 1
            save_as_pickle("evaluate_net_dataset_cpu%i_%i_%s_%s" % (cpu,i,datetime.datetime.today().strftime("%Y-%m-%d"),\
                                                                     str(winner)),dataset)
        print("Current_net wins ratio %.5f" % (current_wins/num_games))
        save_as_pickle("wins_cpu_%i"  % (cpu),\
                                             {"best_win_ratio": current_wins/num_games, "num_games":num_games})
        logger.info("[CPU %d]: Finished arena games!" % cpu)

def fork_process(arena_obj, num_games, cpu):
    arena_obj.evaluate(num_games, cpu)

def evaluate_nets(args, iteration_1, iteration_2):
    logger.info('Loading nets...')
    current_net = "%s_iter%d.pth.tar" % (args.neural_net_name, iteration_2)
    best_net = "%s_iter%d.pth.tar" % (args.neural_net_name, iteration_1)
    current_net_filename = os.path.join("./model_data/", current_net)
    best_net_filename = os.path.join("./model_data/", best_net)

    logger.info('Current net: %s' % current_net)
    logger.info("Previous (Best net: %s" %best_net)

    current_cnet = ConnectNet()
    best_cnet = ConnectNet()
    cuda = torch.cuda.is_available()
    if cuda:
        current_cnet.cuda()
        best_cnet.cuda()

    if not os.path.isdir("./evaluator_data/"):
        os.mkdir("evaluator_data")

    if args.MCTS_num_processes > 1:
        mp.set_start_method("spawn", force=True)

        current_cnet.share_memory()
        best_cnet.share_memory()
        current_cnet.eval()
        best_cnet.eval()

        checkpoint = torch.load(current_net_filename)
        current_cnet.load_state_dict(checkpoint['state_dict'])
        checkpoint = torch.load(best_net_filename)
        best_cnet.load_state_dict(checkpoint['state_dict'])

        processes = []
        if args.MCTS_num_processes > mp.cpu_count():
            num_processes = mp.cpu_count()
            logger.info("Required number of processes exceed number of CPUs! Setting MCTS_num_processes to %d" % num_processes)
        else:
            num_processes = args.MCTS_num_processes
        logger.info("Spawning %d processes..." % num_processes)
        with torch.no_grad():
            for i in range(num_processes):
                p = mp.Process(target=fork_process,args=(arena(current_cnet,best_cnet), args.num_evaluator_games, i))
                p.start()
                processes.append(p)
            for p in processes:
                p.join()

        wins_ratio = 0.0
        for i in range(num_processes):
            stats = load_pickle("wins_cpu_%i" % (i))
            wins_ratio += stats['best_win_ratio']
        wins_ratio = wins_ratio/num_processes
        if wins_ratio >= 0.55:
            return iteration_2
        else :
            return iteration_1

    elif args.MCTS_num_processes == 1:
        current_cnet.eval()
        best_cnet.eval()
        checkpoint = torch.load(current_net_filename)
        current_cnet.load_state_dict(checkpoint['state_dict'])
        checkpoint = torch.load(best_net_filename)
        best_cnet.load_state_dict(checkpoint['state_dict'])
        arena1 = arena(current_cnet=current_cnet, best_cnet=best_cnet)
        arena1.evaluate(num_games=args.num_evaluator_games, cpu=0)

        stats = load_pickle('wins_cpu_%i' % (0))
        if stats["best_win_ratio"] >= 0.55:
            return iteration_2
        else:
            return iteration_1
        
def load_state(net, optimizer, scheduler, args, iteration, new_optim_state=True):
    base_path = "./model-data/"
    checkpoint_path = os.path.join(base_path, "%s_iter%d.pth.tar" % (args.neural_net_name, iteration))
    start_epoch, checkpoint = 0, None
    if os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
    if checkpoint != None:
        if (len(checkpoint) == 1) or (new_optim_state == True):
            net.load_state_dict(checkpoint['state_dict'])
            logger.info("Loaded checkpoint model %s." % checkpoint_path)
        else:
            start_epoch = checkpoint['epoch']
            net.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            scheduler.load_state_dict(checkpoint['scheduler'])
            logger.info('Loaded checkpoint model %s, and optimizer, scheduler.' % checkpoint_path)
    return start_epoch

def load_results(iteration):
    losses_path = "./model_data/losses_per_epoch_iter%d.pkl" % iteration
    if os.path.exists(losses_path):
        losses_per_epoch = load_pickle("losses_per_epoch_iter%d.pkl" % iteration)
        logger.info("Loaded results buffer")
    else:
        losses_per_epoch = []
    return losses_per_epoch


In [21]:
class board_data(Dataset):
    def __init__(self, dataset):  # dataset = np.array of (s, p, v)
        self.X = dataset[:, 0]
        self.y_p, self.y_v = dataset[:, 1], dataset[:, 2]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return np.int64(self.X[idx]), self.y_p[idx], self.y_v[idx]


In [ ]:
import os
import torch.optim as optim

args = {"iteration":0,
       "total_iterations":100,
       "MCTS_num_processes":1,
       "num_games_per_MCTS_process":50,
       "temperature_MCTS":1.1,
       "num_evaluator_games":10,
       "neural_net_name":"tictactoe",
       "batch_size":32,
       "num_epochs":30,
       "lr":0.01,
       "gradient_acc_steps":1.,
       "max_norm":1.}

class Dict2Obj(object):
    """
    Turns a dictionary into a class
    """
    #----------------------------------------------------------------------
    def __init__(self, dictionary):
        """Constructor"""
        for key in dictionary:
            setattr(self, key, dictionary[key])

args = Dict2Obj(args)

for i in range(args.iteration, args.total_iterations):
    run_MCTS(args, 0, i)
    train_connectnet(args, iteration=i, new_optim_state=True)
    if i >= 1:
        winner = evaluate_nets(args, iteration_1=i, iteration_2=i + 1)
        counts = 0
        while (winner != (i+1)):
            logger.info("Trained net didn't perform better, generating more MCTS games for retraining...")
            run_MCTS(args, start_idx=(counts + 1)*args.num_games_per_MCTS_process, iteration=i)
            counts += 1
            train_connectnet(args, iteration=i, new_optim_state=True)
            winner = evaluate_nets(args,iteration_1=i, iteration_2=i + 1)

[I 200412 12:37:37 <ipython-input-18-8fbaffc5cd76>:9] Preparing model for MCTS...
[I 200412 12:37:37 <ipython-input-18-8fbaffc5cd76>:17] Loaded ./model_data/tictactoe_iter5.pth.tar model.
/usr/local/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1340: UserWarning:

nn.functional.tanh is deprecated. Use torch.tanh instead.

[I 200412 12:37:40 <ipython-input-18-8fbaffc5cd76>:24] Finished MCTS!
[I 200412 12:37:40 <ipython-input-19-601145dc3411>:84] Loading training data...
[I 200412 12:37:40 <ipython-input-19-601145dc3411>:92] Loaded data from ./datasets/iter_5/.
[I 200412 12:37:40 <ipython-input-19-601145dc3411>:12] Starting training process...


the length of the train loader is1
Update step size: 1
3.0447428226470947
Update step size: 1
[Iteration 5] Process ID: 11022 [Epoch: 1,    32/ 23 points] total loss per batch: 3.045
Policy (actual, predicted): 8 1
Policy data: tensor([0.0000, 0.0000, 0.1429, 0.0000, 0.2041, 0.1224, 0.1429, 0.0000, 0.3878])
Policy pred: tensor([0.1027, 0.1934, 0.1231, 0.1487, 0.1125, 0.0408, 0.1054, 0.0339, 0.1396],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 -0.13240526616573334
 
2.5238564014434814
Update step size: 1
[Iteration 5] Process ID: 11022 [Epoch: 2,    32/ 23 points] total loss per batch: 2.524
Policy (actual, predicted): 7 2
Policy data: tensor([0.0000, 0.0000, 0.0000, 0.0204, 0.0204, 0.1429, 0.0000, 0.8163, 0.0000])
Policy pred: tensor([0.0416, 0.1014, 0.2810, 0.0338, 0.0387, 0.2393, 0.0401, 0.1623, 0.0618],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 0.0 0.17221017181873322
 
2.525850296020508
Update step size: 1
[Iteration 5] Process ID: 11022 [Epoch

1.2102251052856445
Update step size: 1
[Iteration 5] Process ID: 11022 [Epoch: 21,    32/ 23 points] total loss per batch: 1.210
Policy (actual, predicted): 8 8
Policy data: tensor([0.0816, 0.0000, 0.1429, 0.2041, 0.0612, 0.0816, 0.1633, 0.0000, 0.2653])
Policy pred: tensor([0.0444, 0.0139, 0.1712, 0.1997, 0.0249, 0.0949, 0.1238, 0.0023, 0.3249],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 0.9968484044075012
 
1.1999138593673706
Update step size: 1
[Iteration 5] Process ID: 11022 [Epoch: 22,    32/ 23 points] total loss per batch: 1.200
Policy (actual, predicted): 1 1
Policy data: tensor([0.0408, 0.3878, 0.1020, 0.0612, 0.0612, 0.2245, 0.0204, 0.0000, 0.1020])
Policy pred: tensor([0.0424, 0.3552, 0.0957, 0.0564, 0.0599, 0.2982, 0.0124, 0.0029, 0.0769],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 0.9890971779823303
 
1.1941745281219482
Update step size: 1
[Iteration 5] Process ID: 11022 [Epoch: 23,    32/ 23 points] total loss per batch: 1.194
Pol

[I 200412 12:37:43 <ipython-input-19-601145dc3411>:65] Finished Training!


1.1772958040237427
Update step size: 1
[Iteration 5] Process ID: 11022 [Epoch: 30,    32/ 23 points] total loss per batch: 1.177
Policy (actual, predicted): 8 8
Policy data: tensor([0.0000, 0.0000, 0.1429, 0.2653, 0.1837, 0.0612, 0.0612, 0.0000, 0.2857])
Policy pred: tensor([3.4993e-03, 5.8596e-03, 1.5484e-01, 2.6661e-01, 1.7885e-01, 6.2104e-02,
        5.9502e-02, 1.2115e-04, 2.6860e-01], grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 0.9995348453521729
 


...

the length of the train loader is1
Update step size: 1
2.6539390087127686
Update step size: 1
[Iteration 8] Process ID: 11022 [Epoch: 1,    32/ 23 points] total loss per batch: 2.654
Policy (actual, predicted): 1 6
Policy data: tensor([0.2245, 0.4286, 0.0000, 0.0000, 0.0204, 0.0204, 0.3061, 0.0000, 0.0000])
Policy pred: tensor([0.0839, 0.1200, 0.0721, 0.0716, 0.1134, 0.0718, 0.2079, 0.1720, 0.0874],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 -0.09754529595375061
 
2.439531087875366
Update step size: 1
[Iteration 8] Process ID: 11022 [Epoch: 2,    32/ 23 points] total loss per batch: 2.440
Policy (actual, predicted): 7 3
Policy data: tensor([0.1837, 0.0204, 0.0612, 0.1837, 0.0408, 0.1429, 0.0612, 0.3061, 0.0000])
Policy pred: tensor([0.1828, 0.0817, 0.0348, 0.3660, 0.0202, 0.0758, 0.0524, 0.1564, 0.0299],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 0.0 0.3421141803264618
 
2.8703229427337646
Update step size: 1
[Iteration 8] Process ID: 11022 [Epoch:

1.1475825309753418
Update step size: 1
[Iteration 8] Process ID: 11022 [Epoch: 21,    32/ 23 points] total loss per batch: 1.148
Policy (actual, predicted): 7 7
Policy data: tensor([0.0000, 0.2245, 0.0000, 0.2653, 0.0000, 0.0204, 0.0000, 0.4898, 0.0000])
Policy pred: tensor([0.0034, 0.1819, 0.0274, 0.2996, 0.0026, 0.0277, 0.0174, 0.4362, 0.0037],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 0.9621239900588989
 
1.1561479568481445
Update step size: 1
[Iteration 8] Process ID: 11022 [Epoch: 22,    32/ 23 points] total loss per batch: 1.156
Policy (actual, predicted): 7 7
Policy data: tensor([0.1837, 0.0204, 0.0612, 0.1837, 0.0408, 0.1429, 0.0612, 0.3061, 0.0000])
Policy pred: tensor([0.2130, 0.0312, 0.0554, 0.1592, 0.0421, 0.1267, 0.0556, 0.3159, 0.0010],
       grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 0.7592387199401855
 
1.135990858078003
Update step size: 1
[Iteration 8] Process ID: 11022 [Epoch: 23,    32/ 23 points] total loss per batch: 1.136
Poli

[I 200412 12:40:14 <ipython-input-19-601145dc3411>:65] Finished Training!


1.1772570610046387
Update step size: 1
[Iteration 8] Process ID: 11022 [Epoch: 30,    32/ 23 points] total loss per batch: 1.177
Policy (actual, predicted): 0 0
Policy data: tensor([0.3673, 0.0408, 0.2857, 0.0000, 0.0612, 0.1633, 0.0816, 0.0000, 0.0000])
Policy pred: tensor([4.0164e-01, 4.7254e-02, 2.4292e-01, 3.2748e-03, 6.6163e-02, 1.4805e-01,
        8.7876e-02, 2.8152e-03, 6.2203e-06], grad_fn=<SelectBackward>)
Value (actual, predicted): 1.0 0.9929296970367432
 


[I 200412 12:40:14 <ipython-input-20-615e5088c2dc>:71] Loading nets...
[I 200412 12:40:14 <ipython-input-20-615e5088c2dc>:77] Current net: tictactoe_iter9.pth.tar
[I 200412 12:40:14 <ipython-input-20-615e5088c2dc>:78] Previous (Best net: tictactoe_iter8.pth.tar


## Visualiser la loss

In [ ]:
import os
import matplotlib.pyplot as plt

def load_results(iteration):
    losses_path = "./model_data/losses_per_epoch_iter%d.pkl" % iteration
    if os.path.exists(losses_path):
        losses_per_epoch = load_pickle("losses_per_epoch_iter%d.pkl" % iteration)
        logger.info("Loaded results buffer")
    else:
        losses_per_epoch = []
    return losses_per_epoch

losses_per_epoch = load_results(1 + 1)
fig = plt.figure()
ax = fig.add_subplot(222)
ax.scatter([e for e in range(0, (len(losses_per_epoch) ))], )
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss per batch')

ax.set_title('loss vs Epoch')


## Jouer contre l'ia

In [ ]:
import ast 

def play_game(net):
    # Asks human what he/she wanna play as
    white = None;
    black = None
    while (True):
        play_as = input("What do you wanna play as? (\"O\"/\"X\")? Note: \"O\" starts first, \"X\" starts second\n")
        if play_as == "O":
            black = net;
            break
        elif play_as == "X":
            white = net;
            break
        else:
            print("I didn't get that.")
    current_board = Morpion_board()
    checkmate = False
    dataset = []
    value = 0;
    t = 0.1;
    moves_count = 0
    while checkmate == False and current_board.actions() != []:
        if moves_count <= 5:
            t = 1
        else:
            t = 0.1
        moves_count += 1
        dataset.append(copy.deepcopy(encode_board(current_board)))
        print(current_board.board);
        print(" ")
        if current_board.player == 0:
            if white != None:
                print("AI is thinking........")
                root = UCT_search(current_board, 777, white, t)
                policy = get_policy(root, t)
                current_board = do_decode_n_move_pieces(current_board, \
                                                        int(np.random.choice(np.array(range(9)), \
                                                                             p=policy)))
            else:
                while (True):
                    row, col = ast.literal_eval(
                        input("Which tuple do you wanna drop your piece? (Enter row, column as [(1,3),(1,3)])\n"))
                    current_board.draw_sign([int(row) - 1, int(col) - 1])
                    break
        elif current_board.player == 1:
            if black != None:
                print("AI is thinking.............")
                root = UCT_search(current_board, 333, black, t)
                policy = get_policy(root, t)
                current_board = do_decode_n_move_pieces(current_board, \
                                                        int(np.random.choice(np.array(range(9)), \
                                                                             p=policy)))
            else:
                while (True):
                    row, col = ast.literal_eval(
                        input("Which tuple do you wanna drop your piece? (Enter row, column as [(1,3),(1,3)])\n"))
                    current_board.draw_sign([int(row) - 1, int(col) - 1])
                    break
        # decode move and move piece(s)
        if current_board.check_winner() == True:  # someone wins
            if current_board.player == 0:  # black wins
                value = -1
            elif current_board.player == 1:  # white wins
                value = 1
            checkmate = True
    dataset.append(encode_board(current_board))
    print(current_board.board);
    print(" ")
    if value == 1:
        if play_as == "O":
            dataset.append(f"AI as black wins");
            print("YOU LOSE!!!!!!!")
        else:
            dataset.append(f"Human as black wins");
            print("YOU WIN!!!!!!!")
        return "black", dataset
    elif value == -1:
        if play_as == "O":
            dataset.append(f"Human as white wins");
            print("YOU WIN!!!!!!!!!!!")
        else:
            dataset.append(f"AI as white wins");
            print("YOU LOSE!!!!!!!")
        return "white", dataset
    else:
        dataset.append("Nobody wins");
        print("DRAW!!!!!")
        return None, dataset

In [ ]:
best_net = "tictactoe_iter5.pth.tar"
best_net_filename = os.path.join("./model_data/", \
                                 best_net)
best_cnet = ConnectNet()
# cuda = torch.cuda.is_available()
# if cuda:
#     best_cnet.cuda()
best_cnet.eval()
checkpoint = torch.load(best_net_filename)
best_cnet.load_state_dict(checkpoint['state_dict'])
play_again = True
while (play_again == True):
    play_game(best_cnet)
    while (True):
        again = input("Do you wanna play again? (Y/N)\n")
        if again.lower() in ["y", "n"]:
            if again.lower() == "n":
                play_again = False;
                break
            else:
                break